In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("LANGSMITH_API_KEY: \n")

In [2]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY: \n")

In [3]:
from langchain_tavily import TavilySearch

tavily_api_key = os.environ.get("TAVILY_API_KEY")
if not tavily_api_key:
	raise ValueError("TAVILY_API_KEY environment variable is not set or is empty. Please set it before running this cell.")

search = TavilySearch(max_results=2, tavily_api_key=tavily_api_key)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco, CA', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1757320961, 'localtime': '2025-09-08 01:42'}, 'current': {'last_updated_epoch': 1757320200, 'last_updated': '2025-09-08 01:30', 'temp_c': 16.7, 'temp_f': 62.1, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 5.1, 'wind_kph': 8.3, 'wind_degree': 247, 'wind_dir': 'WSW', 'pressure_mb': 1019.0, 'pressure_in': 30.08, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 90, 'cloud': 25, 'feelslike_c': 16.7, 'feelslike_f': 62.1, 'windchill_c': 15.6, 'windchill_f': 60.1, 'heatindex_c': 15.6, 'heatindex_f': 60.1, 'dewpoint_c': 14.9, 'dewp

In [5]:
from langchain.chat_models import init_chat_model

# Initialize Ollama chat model
model = init_chat_model("llama3", model_provider="ollama")

# Send a prompt
response = model.invoke("Write a short poem about autumn.")
print(response.content)

As summer's warmth begins to fade,
Autumn's hue of gold is made,
The trees don fiery crowns,
And leaves rustle with gentle sounds.

The air is crisp, the breeze is cool,
Nature's final dance, a rustic rule,
The scent of woodsmoke fills the air,
As autumn's chilliness is shared.

The sun sets low, the stars appear,
A season's swan song, whispered clear,
Farewell summer, hello fall,
Autumn's beauty stands tall.


In [25]:

from langchain.tools import Tool

# Wrap TavilySearch in a Tool
tavily_tool = Tool(
    name="TavilySearch",
    func=lambda query: search.invoke(query),
    description="Use this tool to search the web for current information."
)


# Tool for generating text with the LLM
llm_tool = Tool(
    name="TextGenerator",
    func=lambda prompt: model.invoke(prompt).content,
    description="Use this tool to generate text, summaries, poems, or answer questions."
)


# Combine both tools
tools = [tavily_tool, llm_tool]



In [26]:

from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType
agent = initialize_agent(
    tools=tools,
    llm=model,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


In [27]:
response = agent.invoke("What's the weather like in San Francisco and write a haiku about it.")
print(response)



> Entering new AgentExecutor chain...
Thought: To find out the current weather in San Francisco, I should use TavilySearch.

Action: TavilySearch
Action Input: "What is the current weather in San Francisco?"
Observation: {'query': 'What is the current weather in San Francisco?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1757325212, 'localtime': '2025-09-08 02:53'}, 'current': {'last_updated_epoch': 1757324700, 'last_updated': '2025-09-08 02:45', 'temp_c': 16.1, 'temp_f': 61.0, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 5.4, 'wind_kph': 8.6, 'wind_degree': 241, 'wind_dir': 'WSW', 'pressure_mb': 1019.0,